# 🏥 Đánh Giá Mô Hình Medical Chatbot - PhoBERT RAG

**Notebook này đánh giá hiệu suất của hệ thống Medical Chatbot sử dụng:**
- **PhoBERT** (vinai/phobert-base) cho embedding tiếng Việt
- **Hybrid Search** (BM25 + Vector Search)
- **RAG** (Retrieval-Augmented Generation) với GPT-4

## 📊 Các Metrics Đánh Giá

### 1. **Retrieval Metrics** (Đánh giá khả năng tìm kiếm)
- **Precision@K**: Tỷ lệ kết quả đúng trong top K kết quả
- **Recall@K**: Tỷ lệ tìm được tài liệu đúng trong top K
- **MRR (Mean Reciprocal Rank)**: Vị trí trung bình của kết quả đúng đầu tiên
- **NDCG@K**: Normalized Discounted Cumulative Gain

### 2. **Response Quality Metrics** (Đánh giá chất lượng câu trả lời)
- **BLEU Score**: So sánh n-gram với câu trả lời chuẩn
- **ROUGE Score**: Đo độ chồng lấp với câu trả lời chuẩn
- **BERTScore**: Đo semantic similarity bằng BERT
- **Semantic Similarity**: Cosine similarity giữa embeddings

### 3. **Medical Accuracy Metrics** (Đánh giá độ chính xác y tế)
- **Medical Entity Accuracy**: Tỷ lệ thực thể y tế (bệnh, thuốc) đúng
- **Factual Consistency**: Độ nhất quán với nguồn
- **Hallucination Rate**: Tỷ lệ thông tin sai/không có trong nguồn

### 4. **System Performance**
- **Response Time**: Thời gian phản hồi
- **Token Usage**: Số token GPT sử dụng
- **Cost Estimation**: Ước tính chi phí API

---

## 🚀 Hướng Dẫn Sử Dụng

1. **Upload test dataset** (CSV/JSON) với format:
   ```json
   {
     "question": "Sốt xuất huyết có triệu chứng gì?",
     "expected_answer": "Sốt cao, đau đầu, đau khớp...",
     "relevant_doc_ids": ["doc_123", "doc_456"]
   }
   ```

2. **Cấu hình API keys** trong phần Setup

3. **Chạy từng cell** để đánh giá

4. **Xem kết quả** ở cuối notebook (charts + summary table)


## 📦 1. Cài Đặt Dependencies

In [ ]:
!pip install -q transformers torch chromadb openai python-dotenv
!pip install -q nltk rouge-score bert-score sacrebleu
!pip install -q scikit-learn pandas matplotlib seaborn tqdm
!pip install -q sentence-transformers  # For BERTScore and reranking

print("✅ All packages installed successfully!")

## 📚 2. Import Libraries

In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict, Tuple, Any
from tqdm.auto import tqdm

# NLP Libraries
import torch
from transformers import AutoModel, AutoTokenizer
from openai import OpenAI

# Evaluation Metrics
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK data
import nltk
nltk.download('punkt', quiet=True)
nltk.download('wordnet', quiet=True)

# Plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'DejaVu Sans'

print("✅ Libraries imported successfully!")

## ⚙️ 3. Configuration & API Setup

In [ ]:
# ==================== CẤU HÌNH API ====================
# Nhập API keys của bạn
OPENAI_API_KEY = "your-openai-api-key-here"  # Thay bằng key thật

# Hoặc upload file .env và uncomment dòng dưới:
# from google.colab import files
# uploaded = files.upload()  # Upload file .env
# from dotenv import load_dotenv
# load_dotenv()
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)

# ==================== CẤU HÌNH MÔ HÌNH ====================
PHOBERT_MODEL = "vinai/phobert-base"  # Giống với backend của bạn
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🔧 Device: {DEVICE}")
print(f"🤖 PhoBERT Model: {PHOBERT_MODEL}")
print(f"✅ Configuration completed!")

## 🧠 4. PhoBERT Embedding Class (Giống Backend)

In [ ]:
class PhoBERTEmbedding:
    """PhoBERT embedding function - giống với backend"""
    
    def __init__(self, model_name=PHOBERT_MODEL, device=DEVICE, max_length=256):
        print(f"Loading PhoBERT model: {model_name}...")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.to(device)
        self.model.eval()
        self.device = device
        self.max_length = max_length
        print(f"✅ PhoBERT loaded on {device}")
    
    def encode(self, texts: List[str]) -> np.ndarray:
        """Generate embeddings for texts"""
        if isinstance(texts, str):
            texts = [texts]
        
        encoded = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors='pt',
            max_length=self.max_length
        )
        
        encoded = {k: v.to(self.device) for k, v in encoded.items()}
        
        with torch.no_grad():
            output = self.model(**encoded)
        
        # Mean pooling
        embeddings = self._mean_pooling(output.last_hidden_state, encoded['attention_mask'])
        return embeddings.cpu().numpy()
    
    def _mean_pooling(self, token_embeddings, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

# Initialize PhoBERT
phobert = PhoBERTEmbedding()

## 📂 5. Load Test Dataset

**Upload file CSV hoặc JSON với format:**

```csv
question,expected_answer,relevant_doc_ids
"Sốt xuất huyết có triệu chứng gì?","Sốt cao đột ngột, đau đầu, đau khớp, xuất huyết","doc_123,doc_456"
```

Hoặc tạo test data mẫu:

In [ ]:
# ==================== OPTION 1: Upload File ====================
# Uncomment để upload file từ máy tính
# from google.colab import files
# uploaded = files.upload()
# test_data_file = list(uploaded.keys())[0]
# test_df = pd.read_csv(test_data_file) if test_data_file.endswith('.csv') else pd.read_json(test_data_file)

# ==================== OPTION 2: Sample Data ====================
# Tạo test data mẫu (thay bằng data thật của bạn)
test_data = [
    {
        "question": "Sốt xuất huyết có triệu chứng gì?",
        "expected_answer": "Sốt cao đột ngột (39-40°C), đau đầu dữ dội, đau mỏi cơ và khớp, buồn nôn, nôn, xuất hiện ban xuất huyết dưới da.",
        "relevant_doc_ids": ["dengue_001"],
        "category": "symptoms"
    },
    {
        "question": "Cách điều trị sốt xuất huyết?",
        "expected_answer": "Nghỉ ngơi tuyệt đối, uống nhiều nước, hạ sốt bằng paracetamol, theo dõi số lượng tiểu cầu. Không dùng aspirin hay ibuprofen. Nhập viện nếu xuất huyết nặng.",
        "relevant_doc_ids": ["dengue_001", "dengue_treatment_001"],
        "category": "treatment"
    },
    {
        "question": "Viêm gan B lây qua đường nào?",
        "expected_answer": "Lây qua đường máu (kim tiêm, dụng cụ y tế), quan hệ tình dục không an toàn, từ mẹ sang con khi sinh.",
        "relevant_doc_ids": ["hepatitis_b_001"],
        "category": "transmission"
    },
    {
        "question": "Paracetamol uống liều bao nhiêu?",
        "expected_answer": "Người lớn: 500-1000mg mỗi lần, cách 4-6 giờ, tối đa 4000mg/ngày. Trẻ em: 10-15mg/kg cân nặng mỗi lần.",
        "relevant_doc_ids": ["paracetamol_001"],
        "category": "medication"
    },
    {
        "question": "Phòng ngừa COVID-19 như thế nào?",
        "expected_answer": "Tiêm vắc-xin đầy đủ, đeo khẩu trang nơi đông người, rửa tay thường xuyên, giữ khoảng cách an toàn, tránh tụ tập đông người.",
        "relevant_doc_ids": ["covid19_prevention_001"],
        "category": "prevention"
    }
]

test_df = pd.DataFrame(test_data)

print(f"✅ Loaded {len(test_df)} test questions")
print(f"\nCategories: {test_df['category'].value_counts().to_dict()}")
test_df.head()

## 🔧 6. Mock RAG System (Simulating Your Backend)

**Lưu ý:** Đây là mock system để test. Trong thực tế, bạn nên:
1. Connect trực tiếp đến backend API của bạn
2. Hoặc load ChromaDB từ backend để test offline

In [ ]:
class MockMedicalRAG:
    """Mock RAG system - thay bằng API call thật hoặc load ChromaDB"""
    
    def __init__(self, phobert_model):
        self.phobert = phobert_model
        self.documents = self._load_mock_documents()
        self.doc_embeddings = self._create_embeddings()
    
    def _load_mock_documents(self) -> List[Dict]:
        """Load mock medical documents (thay bằng ChromaDB thật)"""
        return [
            {
                "id": "dengue_001",
                "disease_name": "Sốt xuất huyết",
                "symptoms": "Sốt cao đột ngột 39-40°C, đau đầu dữ dội, đau mỏi cơ khớp, buồn nôn, ban xuất huyết",
                "treatment": "Nghỉ ngơi, uống nhiều nước, paracetamol hạ sốt, theo dõi tiểu cầu",
                "prevention": "Diệt muỗi, không để nước đọng"
            },
            {
                "id": "dengue_treatment_001",
                "disease_name": "Điều trị sốt xuất huyết",
                "treatment": "Không dùng aspirin, ibuprofen. Chỉ dùng paracetamol. Nhập viện nếu xuất huyết nặng, tiểu cầu dưới 50,000",
            },
            {
                "id": "hepatitis_b_001",
                "disease_name": "Viêm gan B",
                "symptoms": "Mệt mỏi, vàng da, nước tiểu sẫm màu, đau bụng",
                "transmission": "Lây qua máu, quan hệ tình dục, mẹ sang con",
                "prevention": "Tiêm vắc-xin, không dùng chung kim tiêm"
            },
            {
                "id": "paracetamol_001",
                "medication_name": "Paracetamol",
                "dosage": "Người lớn: 500-1000mg/lần, 4-6h/lần, max 4000mg/ngày. Trẻ em: 10-15mg/kg",
                "indication": "Hạ sốt, giảm đau"
            },
            {
                "id": "covid19_prevention_001",
                "disease_name": "COVID-19",
                "prevention": "Tiêm vắc-xin, đeo khẩu trang, rửa tay, giữ khoảng cách",
                "symptoms": "Sốt, ho, khó thở, mất vị giác"
            }
        ]
    
    def _create_embeddings(self) -> np.ndarray:
        """Create embeddings for all documents"""
        texts = []
        for doc in self.documents:
            # Combine all fields into searchable text
            text = " ".join([str(v) for v in doc.values() if v and isinstance(v, str)])
            texts.append(text)
        
        return self.phobert.encode(texts)
    
    def search(self, question: str, top_k: int = 3) -> List[Dict]:
        """Search for relevant documents"""
        # Encode question
        query_embedding = self.phobert.encode([question])
        
        # Calculate cosine similarity
        similarities = cosine_similarity(query_embedding, self.doc_embeddings)[0]
        
        # Get top K
        top_indices = np.argsort(similarities)[::-1][:top_k]
        
        results = []
        for idx in top_indices:
            results.append({
                "id": self.documents[idx]["id"],
                "metadata": self.documents[idx],
                "relevance_score": float(similarities[idx]),
                "distance": 1 - float(similarities[idx])
            })
        
        return results
    
    def generate_answer(self, question: str, search_results: List[Dict]) -> str:
        """Generate answer using GPT (giống backend)"""
        if not search_results:
            return "Xin lỗi, tôi không tìm thấy thông tin phù hợp."
        
        # Prepare context
        context_parts = []
        for idx, result in enumerate(search_results[:3], 1):
            metadata = result['metadata']
            context_parts.append(f"[Nguồn {idx}] {json.dumps(metadata, ensure_ascii=False)}")
        
        context = "\n".join(context_parts)
        
        # Generate with GPT
        try:
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "Bạn là bác sĩ AI. Trả lời dựa trên nguồn được cung cấp. Không tự suy luận."},
                    {"role": "user", "content": f"Nguồn:\n{context}\n\nCâu hỏi: {question}\n\nTrả lời:"}
                ],
                temperature=0.3,
                max_tokens=300
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            return f"Error: {str(e)}"

# Initialize mock RAG system
rag_system = MockMedicalRAG(phobert)
print(f"✅ Mock RAG system initialized with {len(rag_system.documents)} documents")

## 📊 7. Evaluation Functions

In [ ]:
# ==================== RETRIEVAL METRICS ====================

def calculate_precision_at_k(retrieved_ids: List[str], relevant_ids: List[str], k: int) -> float:
    """Precision@K: Tỷ lệ tài liệu đúng trong top K"""
    retrieved_k = retrieved_ids[:k]
    relevant_set = set(relevant_ids)
    correct = sum(1 for doc_id in retrieved_k if doc_id in relevant_set)
    return correct / k if k > 0 else 0.0

def calculate_recall_at_k(retrieved_ids: List[str], relevant_ids: List[str], k: int) -> float:
    """Recall@K: Tỷ lệ tìm được tài liệu đúng trong top K"""
    retrieved_k = set(retrieved_ids[:k])
    relevant_set = set(relevant_ids)
    if len(relevant_set) == 0:
        return 0.0
    correct = len(retrieved_k & relevant_set)
    return correct / len(relevant_set)

def calculate_mrr(retrieved_ids: List[str], relevant_ids: List[str]) -> float:
    """Mean Reciprocal Rank: 1 / vị trí của kết quả đúng đầu tiên"""
    relevant_set = set(relevant_ids)
    for i, doc_id in enumerate(retrieved_ids, 1):
        if doc_id in relevant_set:
            return 1.0 / i
    return 0.0

def calculate_ndcg_at_k(retrieved_ids: List[str], relevant_ids: List[str], k: int) -> float:
    """NDCG@K: Normalized Discounted Cumulative Gain"""
    retrieved_k = retrieved_ids[:k]
    relevant_set = set(relevant_ids)
    
    # DCG
    dcg = sum((1 if doc_id in relevant_set else 0) / np.log2(i + 2) 
              for i, doc_id in enumerate(retrieved_k))
    
    # IDCG (ideal DCG)
    ideal_k = min(k, len(relevant_ids))
    idcg = sum(1 / np.log2(i + 2) for i in range(ideal_k))
    
    return dcg / idcg if idcg > 0 else 0.0

# ==================== RESPONSE QUALITY METRICS ====================

def calculate_bleu(generated: str, reference: str) -> float:
    """BLEU Score: N-gram overlap"""
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    smoothing = SmoothingFunction().method1
    return sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing)

def calculate_rouge(generated: str, reference: str) -> Dict[str, float]:
    """ROUGE Scores: ROUGE-1, ROUGE-2, ROUGE-L"""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    scores = scorer.score(reference, generated)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

def calculate_semantic_similarity(generated: str, reference: str, model) -> float:
    """Semantic Similarity using PhoBERT embeddings"""
    emb1 = model.encode([generated])
    emb2 = model.encode([reference])
    return float(cosine_similarity(emb1, emb2)[0][0])

# ==================== MEDICAL ACCURACY METRICS ====================

def extract_medical_entities(text: str) -> set:
    """Extract medical entities (simple keyword-based, có thể dùng NER model)"""
    # Danh sách từ khóa y tế phổ biến (mở rộng theo nhu cầu)
    medical_keywords = [
        'sốt', 'đau', 'viêm', 'nhiễm', 'bệnh', 'thuốc', 'paracetamol', 'aspirin',
        'xuất huyết', 'tiểu cầu', 'gan', 'phổi', 'tim', 'vắc-xin', 'điều trị'
    ]
    text_lower = text.lower()
    return {kw for kw in medical_keywords if kw in text_lower}

def calculate_entity_accuracy(generated: str, reference: str) -> float:
    """Medical Entity Accuracy: Tỷ lệ thực thể y tế đúng"""
    gen_entities = extract_medical_entities(generated)
    ref_entities = extract_medical_entities(reference)
    
    if len(ref_entities) == 0:
        return 1.0 if len(gen_entities) == 0 else 0.0
    
    correct = len(gen_entities & ref_entities)
    return correct / len(ref_entities)

print("✅ Evaluation functions defined!")

## 🚀 8. Run Full Evaluation

In [ ]:
def evaluate_rag_system(test_df: pd.DataFrame, rag_system, phobert_model, k_values=[1, 3, 5]):
    """
    Đánh giá toàn diện hệ thống RAG
    
    Returns:
        results_df: DataFrame với kết quả chi tiết từng câu hỏi
        metrics_summary: Dict với metrics tổng hợp
    """
    results = []
    
    print(f"🔍 Evaluating {len(test_df)} questions...\n")
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
        question = row['question']
        expected_answer = row['expected_answer']
        relevant_doc_ids = row['relevant_doc_ids']
        
        # Measure response time
        start_time = time.time()
        
        # 1. RETRIEVAL
        search_results = rag_system.search(question, top_k=max(k_values))
        retrieved_ids = [r['id'] for r in search_results]
        
        # 2. GENERATION
        generated_answer = rag_system.generate_answer(question, search_results)
        
        response_time = time.time() - start_time
        
        # 3. CALCULATE METRICS
        result = {
            'question': question,
            'expected_answer': expected_answer,
            'generated_answer': generated_answer,
            'retrieved_ids': retrieved_ids,
            'relevant_ids': relevant_doc_ids,
            'response_time': response_time,
        }
        
        # Retrieval metrics
        for k in k_values:
            result[f'precision@{k}'] = calculate_precision_at_k(retrieved_ids, relevant_doc_ids, k)
            result[f'recall@{k}'] = calculate_recall_at_k(retrieved_ids, relevant_doc_ids, k)
            result[f'ndcg@{k}'] = calculate_ndcg_at_k(retrieved_ids, relevant_doc_ids, k)
        
        result['mrr'] = calculate_mrr(retrieved_ids, relevant_doc_ids)
        
        # Response quality metrics
        result['bleu'] = calculate_bleu(generated_answer, expected_answer)
        rouge_scores = calculate_rouge(generated_answer, expected_answer)
        result.update(rouge_scores)
        result['semantic_similarity'] = calculate_semantic_similarity(
            generated_answer, expected_answer, phobert_model
        )
        
        # Medical accuracy
        result['entity_accuracy'] = calculate_entity_accuracy(generated_answer, expected_answer)
        
        results.append(result)
    
    results_df = pd.DataFrame(results)
    
    # Calculate summary statistics
    metrics_summary = {}
    
    # Retrieval metrics
    for k in k_values:
        metrics_summary[f'Precision@{k}'] = results_df[f'precision@{k}'].mean()
        metrics_summary[f'Recall@{k}'] = results_df[f'recall@{k}'].mean()
        metrics_summary[f'NDCG@{k}'] = results_df[f'ndcg@{k}'].mean()
    
    metrics_summary['MRR'] = results_df['mrr'].mean()
    
    # Response quality
    metrics_summary['BLEU'] = results_df['bleu'].mean()
    metrics_summary['ROUGE-1'] = results_df['rouge1'].mean()
    metrics_summary['ROUGE-2'] = results_df['rouge2'].mean()
    metrics_summary['ROUGE-L'] = results_df['rougeL'].mean()
    metrics_summary['Semantic Similarity'] = results_df['semantic_similarity'].mean()
    
    # Medical accuracy
    metrics_summary['Entity Accuracy'] = results_df['entity_accuracy'].mean()
    
    # Performance
    metrics_summary['Avg Response Time (s)'] = results_df['response_time'].mean()
    
    return results_df, metrics_summary

# RUN EVALUATION
results_df, metrics_summary = evaluate_rag_system(test_df, rag_system, phobert)

print("\n" + "="*60)
print("📊 EVALUATION RESULTS")
print("="*60)
for metric, value in metrics_summary.items():
    print(f"{metric:.<40} {value:.4f}")
print("="*60)

## 📈 9. Visualization

In [ ]:
# ==================== METRICS COMPARISON ====================
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Retrieval Metrics
retrieval_metrics = {
    'Precision@1': metrics_summary['Precision@1'],
    'Precision@3': metrics_summary['Precision@3'],
    'Recall@1': metrics_summary['Recall@1'],
    'Recall@3': metrics_summary['Recall@3'],
    'MRR': metrics_summary['MRR'],
    'NDCG@3': metrics_summary['NDCG@3']
}
axes[0, 0].bar(retrieval_metrics.keys(), retrieval_metrics.values(), color='skyblue')
axes[0, 0].set_title('Retrieval Metrics', fontsize=14, fontweight='bold')
axes[0, 0].set_ylabel('Score')
axes[0, 0].set_ylim(0, 1)
axes[0, 0].grid(axis='y', alpha=0.3)
plt.setp(axes[0, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')

# 2. Response Quality Metrics
quality_metrics = {
    'BLEU': metrics_summary['BLEU'],
    'ROUGE-1': metrics_summary['ROUGE-1'],
    'ROUGE-2': metrics_summary['ROUGE-2'],
    'ROUGE-L': metrics_summary['ROUGE-L'],
    'Semantic Sim': metrics_summary['Semantic Similarity']
}
axes[0, 1].bar(quality_metrics.keys(), quality_metrics.values(), color='lightcoral')
axes[0, 1].set_title('Response Quality Metrics', fontsize=14, fontweight='bold')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_ylim(0, 1)
axes[0, 1].grid(axis='y', alpha=0.3)
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45, ha='right')

# 3. Per-Question Performance
results_df['question_short'] = results_df['question'].str[:30] + '...'
x_pos = np.arange(len(results_df))
axes[1, 0].bar(x_pos, results_df['semantic_similarity'], alpha=0.7, label='Semantic Sim', color='green')
axes[1, 0].bar(x_pos, results_df['bleu'], alpha=0.7, label='BLEU', color='orange')
axes[1, 0].set_title('Per-Question Performance', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Question Index')
axes[1, 0].set_ylabel('Score')
axes[1, 0].legend()
axes[1, 0].grid(axis='y', alpha=0.3)

# 4. Response Time Distribution
axes[1, 1].hist(results_df['response_time'], bins=10, color='purple', alpha=0.7, edgecolor='black')
axes[1, 1].axvline(results_df['response_time'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {results_df["response_time"].mean():.2f}s')
axes[1, 1].set_title('Response Time Distribution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Response Time (seconds)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].legend()
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# ==================== DETAILED RESULTS TABLE ====================
print("\n📋 DETAILED RESULTS (First 3 Questions)\n")
display_cols = ['question', 'precision@3', 'recall@3', 'bleu', 'semantic_similarity', 'entity_accuracy']
print(results_df[display_cols].head(3).to_string(index=False))

## 💾 10. Export Results

In [ ]:
# Export to CSV
results_df.to_csv('evaluation_results.csv', index=False, encoding='utf-8-sig')

# Export summary to JSON
with open('metrics_summary.json', 'w', encoding='utf-8') as f:
    json.dump(metrics_summary, f, ensure_ascii=False, indent=2)

print("✅ Results exported:")
print("   - evaluation_results.csv")
print("   - metrics_summary.json")

# Download files (for Colab)
try:
    from google.colab import files
    files.download('evaluation_results.csv')
    files.download('metrics_summary.json')
except:
    print("\n(Not in Colab - files saved locally)")

## 📖 11. Interpretation Guide

### 🎯 Cách Đánh Giá Mô Hình Tốt Hay Không

#### **A. Retrieval Metrics (Khả năng tìm kiếm)**

| Metric | Tốt | Trung Bình | Yếu | Ý Nghĩa |
|--------|-----|------------|-----|----------|
| **Precision@3** | ≥ 0.8 | 0.5-0.8 | < 0.5 | Tỷ lệ tài liệu đúng trong top 3 |
| **Recall@3** | ≥ 0.9 | 0.6-0.9 | < 0.6 | Tìm được bao nhiêu % tài liệu liên quan |
| **MRR** | ≥ 0.8 | 0.5-0.8 | < 0.5 | Tài liệu đúng có ở vị trí cao không |
| **NDCG@3** | ≥ 0.85 | 0.6-0.85 | < 0.6 | Chất lượng ranking tổng thể |

**Nếu Retrieval kém:**
- ✅ Cải thiện embedding (fine-tune PhoBERT)
- ✅ Tăng trọng số BM25 (keyword matching)
- ✅ Thêm query expansion
- ✅ Reranking với Cross-Encoder

---

#### **B. Response Quality Metrics (Chất lượng câu trả lời)**

| Metric | Tốt | Trung Bình | Yếu | Ý Nghĩa |
|--------|-----|------------|-----|----------|
| **BLEU** | ≥ 0.4 | 0.2-0.4 | < 0.2 | Độ giống về từ ngữ |
| **ROUGE-L** | ≥ 0.5 | 0.3-0.5 | < 0.3 | Độ chồng lấp câu dài |
| **Semantic Similarity** | ≥ 0.75 | 0.6-0.75 | < 0.6 | Độ giống về ý nghĩa (quan trọng nhất!) |
| **Entity Accuracy** | ≥ 0.85 | 0.7-0.85 | < 0.7 | Tỷ lệ thuật ngữ y tế đúng |

**Nếu Response Quality kém:**
- ✅ Cải thiện prompt cho GPT
- ✅ Tăng số lượng context (top K results)
- ✅ Fine-tune GPT với medical data
- ✅ Thêm fact-checking layer

---

#### **C. System Performance**

| Metric | Tốt | Chấp Nhận Được | Yếu |
|--------|-----|----------------|-----|
| **Response Time** | < 2s | 2-5s | > 5s |
| **Token Usage** | < 500 tokens/query | 500-1000 | > 1000 |

---

### 🔍 Ví Dụ Phân Tích

**Kịch bản 1: Retrieval tốt nhưng Response kém**
```
Precision@3: 0.85 ✅
Semantic Similarity: 0.45 ❌
```
→ **Nguyên nhân:** GPT không tổng hợp thông tin tốt  
→ **Giải pháp:** Cải thiện prompt, thêm examples

**Kịch bản 2: Retrieval kém nhưng Response tốt**
```
Precision@3: 0.40 ❌
Semantic Similarity: 0.80 ✅
```
→ **Nguyên nhân:** GPT tự suy luận (hallucination!)  
→ **Giải pháp:** Cải thiện retrieval, giảm temperature GPT

**Kịch bản 3: Cả 2 đều tốt**
```
Precision@3: 0.90 ✅
Semantic Similarity: 0.85 ✅
```
→ **Mô hình đạt chuẩn production!** 🎉

---

### 📝 Checklist Đánh Giá Cuối Cùng

- [ ] **Retrieval Precision@3 ≥ 0.7**
- [ ] **Semantic Similarity ≥ 0.7**
- [ ] **Entity Accuracy ≥ 0.8**
- [ ] **Response Time < 3s**
- [ ] **Không có hallucination nghiêm trọng**

Nếu đạt **4/5 tiêu chí** → Mô hình **TỐT**, có thể deploy!
